In [2]:
%matplotlib inline
from stack_nlp import *
from jupyter_mplsettings import *
matplotlib.rc('font', **font)
from glob import glob

Using TensorFlow backend.


In [5]:
cfg = local_import("./laptop.py")

In [ ]:
conn = sqlite3.connect(cfg.posts_path)

In [8]:
allmetas = sorted(glob(os.path.join(cfg.paths["metas"], "posts_*.hdf5")))
print allmetas

['/home/alex/data/stackexchange/overflow/caches/posts_2008.hdf5', '/home/alex/data/stackexchange/overflow/caches/posts_2009.hdf5', '/home/alex/data/stackexchange/overflow/caches/posts_2010.hdf5', '/home/alex/data/stackexchange/overflow/caches/posts_2011.hdf5', '/home/alex/data/stackexchange/overflow/caches/posts_2012.hdf5', '/home/alex/data/stackexchange/overflow/caches/posts_2013.hdf5', '/home/alex/data/stackexchange/overflow/caches/posts_2014.hdf5', '/home/alex/data/stackexchange/overflow/caches/posts_2015.hdf5', '/home/alex/data/stackexchange/overflow/caches/posts_2016.hdf5', '/home/alex/data/stackexchange/overflow/caches/posts_2017.hdf5']


In [10]:
highscorers = []
for meta_path in allmetas:
    print meta_path
    store_meta = pd.HDFStore(meta_path, "r", complib="blosc", complevel=9)
    smask = store_meta.select_as_coordinates("posts", "Score > 10")
    ids = store_meta.select("posts", where=smask, columns="Id").values
    print "%i high-scorers" % len(ids)
    highscorers.append(ids)

/home/alex/data/stackexchange/overflow/caches/posts_2008.hdf5
47818 high-scorers
/home/alex/data/stackexchange/overflow/caches/posts_2009.hdf5
129569 high-scorers
/home/alex/data/stackexchange/overflow/caches/posts_2010.hdf5
169439 high-scorers
/home/alex/data/stackexchange/overflow/caches/posts_2011.hdf5
203142 high-scorers
/home/alex/data/stackexchange/overflow/caches/posts_2012.hdf5
197105 high-scorers
/home/alex/data/stackexchange/overflow/caches/posts_2013.hdf5
174314 high-scorers
/home/alex/data/stackexchange/overflow/caches/posts_2014.hdf5
110332 high-scorers
/home/alex/data/stackexchange/overflow/caches/posts_2015.hdf5
80620 high-scorers
/home/alex/data/stackexchange/overflow/caches/posts_2016.hdf5
42255 high-scorers
/home/alex/data/stackexchange/overflow/caches/posts_2017.hdf5
8907 high-scorers


In [ ]:
from gensim.parsing import preprocessing

i = 0
for post in YieldDBPosts(conn):
    print post[0]
    print preprocessing.preprocess_string(post[0])
    i += 1
    if i == 3:
        break